In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('customerChurn').getOrCreate()

In [3]:
df = spark.read.csv('customer_churn.csv', inferSchema = True, header = True)

In [4]:
df.show(5)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|  Cynthia Norton|37.0|    

In [5]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
my_cols = df.select(['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites', 'Churn'])

In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler = VectorAssembler(inputCols = ['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'],
                            outputCol = 'features')

In [24]:
output = assembler.transform(df)

In [25]:
final_data = output.select('features', 'churn')

### Model Building

In [10]:
from pyspark.ml.classification import LogisticRegression

In [11]:
from pyspark.ml import Pipeline

In [31]:
log_reg = LogisticRegression(featuresCol = 'features', labelCol = 'churn')

In [32]:
#pipeline = Pipeline(stages = [assembler, log_reg])

In [33]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [34]:
fit_model = log_reg.fit(train_data)

In [35]:
training_sum =  fit_model.summary

In [36]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                645|                645|
|   mean|0.17519379844961241|0.14108527131782947|
| stddev| 0.3804278009855792|0.34837965723425135|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



### Model Evaluation

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [37]:
pred_and_labels = fit_model.evaluate(test_data)

In [38]:
pred_and_labels.predictions.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.85540154502169...|[0.99227395040221...|       0.0|
|[26.0,8939.61,0.0...|    0|[6.59324145067707...|[0.99863227992865...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.02435645978920...|[0.99967271650218...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.62228069611739...|[0.83510942555472...|       0.0|
|[29.0,9378.24,0.0...|    0|[4.9114443435517,...|[0.99269195311583...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [39]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol = 'prediction', labelCol = 'churn')

In [41]:
AUC = my_eval.evaluate(pred_and_labels.predictions)

In [42]:
AUC

0.7001611703446565

### Predict on new data

In [43]:
final_log_model = log_reg.fit(final_data)

In [44]:
new_customers = spark.read.csv('new_customers.csv', inferSchema = True, header = True)

In [46]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [47]:
test_new_customers = assembler.transform(new_customers)

In [48]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [49]:
final_results = final_log_model.transform(test_new_customers)

In [51]:
final_results.select('company', 'prediction').show()

+----------------+----------+
|         company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [52]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         NULL|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                NULL|            NULL|
| stddev|         NULL| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                NULL|            NULL|
|    min|Andrew Mccall|              22.0|            100.0|          